# QA em português usando documentos PDF

Como ilustração, usaremos o livro monumental de Herótodo, *História*, com 770 páginas.

In [1]:
!pip install -q transformers

     |████████████████████████████████| 2.1MB 18.7MB/s 
     |████████████████████████████████| 901kB 55.6MB/s 
     |████████████████████████████████| 3.3MB 55.4MB/s 


In [2]:
import transformers
from transformers import pipeline

modelo = 'pierreguillou/bert-base-cased-squad-v1.1-portuguese'

In [3]:
!pip install pdfplumber

     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 5.6MB 23.8MB/s 
     |████████████████████████████████| 143kB 57.2MB/s 
     |████████████████████████████████| 1.9MB 50.6MB/s 
  Created wheel for pdfplumber: filename=pdfplumber-0.5.27-cp37-none-any.whl size=32071 sha256=e9c45b554b4014bd111a2859d30d86c45ad6c9a7f667cf087977388413f66a6a
  Stored in directory: /root/.cache/pip/wheels/b4/74/fc/f7b3a1a0732209027fb48a5f4392fc40d79970b11c2ba49e71
Successfully built pdfplumber


In [4]:
import pdfplumber

In [5]:
arq = pdfplumber.open('./docs/HERODOTO.pdf')

In [6]:
txt = arq.pages[200].extract_text()
txt

'CXL — Assim que Sabacos deixou o Egito, Anísis (o\ncego) saiu da região pantanosa onde se refugiara e retomou as\nrédeas  do  governo.  Tinha  permanecido  cinqüenta  anos  numa\nilha por ele próprio formada com cinza e terra, pois, quando os\nEgípcios lhe iam levar víveres, cada um de acordo com as suas\nposses, ele lhes pedia um pouco de cinza, que ia acumulando\nem mistura com a terra. Essa ilha só foi encontrada no reinado\nde  Amirteu.  Durante  mais  de  quinhentos  anos,  os  reis  seus\npredecessores  tinham-na  procurado  inutilmente.  Chamam-na\nilha de Helbo e mede dez estádios em qualquer sentido, pois é\nde formato arredondado.\nCXLI  —  Anísis  teve  por  sucessor  um  sacerdote  de\nVulcano  chamado  Setos(25).  Esse  governante  nenhuma\nconsideração  teve  para  com  os  guerreiros,  tratando-os  com\ninteiro desprezo e chegando ao extremo de privá-los dos doze\nalqueires de terra que os reis seus antecessores haviam dado a\ncada um como prêmio. Por isso, quando Senaq

In [7]:
nlp = pipeline('question-answering', model=modelo)

pergunta = 'Quem sucedeu a Anísis?'
resp = nlp(question=pergunta, context=txt)

In [8]:
print(f"Resposta: '{resp['answer']}', score: {round(resp['score'], 4)}, start: {resp['start']}, end: {resp['end']}")

Resposta: 'Setos', score: 0.7311, start: 713, end: 718


In [9]:
txt[713:718]

'Setos'

# Exercício: QA na web

Neste exercício, você vai usar o mesmo modelo do BERT para responder a perguntas sobre um documento de texto da Wikipedia.

O script para acessar e limpar o texto da Wikipedia já foi feito para você. As perguntas, também.

Quais as respostas?

In [12]:
import requests
from bs4 import BeautifulSoup

pagina = requests.get('https://pt.wikipedia.org/wiki/Bossa_nova')
sopa = BeautifulSoup(pagina.text, 'html.parser')

for script in sopa(['script', 'style']):
    script.decompose()

desetiquetado = list(sopa.stripped_strings)
desetiquetado = [i for i in desetiquetado if len(i) > 3]

wiki = ' '.join(desetiquetado)

In [14]:
print(desetiquetado[:5])
print(wiki)

['Bossa nova – Wikipédia, a enciclopédia livre', 'Saltar para o conteúdo', 'Alternar barra lateral', 'Busca', 'Ferramentas pessoais']
Bossa nova – Wikipédia, a enciclopédia livre Saltar para o conteúdo Alternar barra lateral Busca Ferramentas pessoais Não autenticado Discussão Contribuições Criar uma conta Entrar Menu de navegação Navegação Página principal Conteúdo destacado Eventos atuais Esplanada Página aleatória Portais Informar um erro Colaboração Boas-vindas Ajuda Página de testes Portal comunitário Mudanças recentes Manutenção Criar página Páginas novas Contato Donativos Ferramentas Páginas afluentes Alterações relacionadas Carregar ficheiro Páginas especiais Hiperligação permanente Informações da página Citar esta página Elemento Wikidata Imprimir/exportar Criar um livro Descarregar como PDF Versão para impressão Noutros projetos Wikimedia Commons Wikiquote Noutras línguas Afrikaans العربية Asturianu Azərbaycanca Български বাংলা Català کوردی Čeština Cymraeg Dansk Deutsch Ελλην

In [16]:
nlp = pipeline('question-answering', model=modelo)

pergunta = 'Qual foi o prêmio recebido por João Gilberto e Stan Getz?'
resp = nlp(question=pergunta, context=wiki)

In [17]:
resp

{'answer': 'Grammy Award',
 'end': 5052,
 'score': 0.6949346661567688,
 'start': 5040}

In [19]:
pergunta = 'Qual foi o jazzista de maior influência sobre o estilo de canto da bossa-nova?'
resp = nlp(question=pergunta, context=wiki)

In [20]:
resp

{'answer': 'Chet Baker',
 'end': 3769,
 'score': 0.9942364692687988,
 'start': 3759}

In [18]:
pergunta = 'Onde morava Nara Leão?'
resp = nlp(question=pergunta, context=wiki)
resp

{'answer': 'Avenida Atlântica',
 'end': 8129,
 'score': 0.5305925011634827,
 'start': 8112}